# Exploring Pyspark
Exploring Pyspark with NYC Yellow Taxi Data

In [ ]:
#!pip install pyspark

In [ ]:
#import pyspark

Pyspark will not run if Java is not installed on the computer. PySpark 4.x requires Java 17 or 21. PySpark 3.5 works perfectly with Java 8

In [ ]:
#pyspark.__version__

In [ ]:
#pip uninstall pyspark -y

In [ ]:
# Force install pyspark version
#!pip install pyspark==3.5.1

In [1]:
import pyspark

In [2]:
from pyspark.sql.functions import expr,col, regexp_replace, coalesce, lit, when
from pyspark.sql import SparkSession

In [3]:
#Build a Pyspark Session
spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [ ]:
# read csv file in pyspark
df = spark.read.csv(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\csv_trip_data\yellow_tripdata_2024-01.csv'
    ,header=True #sets the first row to be the header
    ,inferSchema = True #Default is string if infer Schema is not specified.Ensures to infer the correct datatype of columns. 
)

In [ ]:
#Another way to read files
df=spark.read.option('header','true').csv(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\csv_trip_data\yellow_tripdata_2024-01.csv'
)

In [ ]:
df.show()

In [ ]:
# Check dataframe type
type(df)

In [ ]:
#Print Schema
df.printSchema()

In [ ]:
# Select columns in pyspark
df.select('VendorID','tpep_pickup_datetime').show()

In [4]:
#Read Parquet file
df_parq= spark.read.parquet(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\yellow_tripdata_2024-01.parquet')

In [5]:
df_parq.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [6]:
df_parq.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [7]:
# Select columns
df_parq.select('VendorID','tpep_pickup_datetime','store_and_fwd_flag').show(5)

+--------+--------------------+------------------+
|VendorID|tpep_pickup_datetime|store_and_fwd_flag|
+--------+--------------------+------------------+
|       2| 2024-01-01 00:57:55|                 N|
|       1| 2024-01-01 00:03:00|                 N|
|       1| 2024-01-01 00:17:06|                 N|
|       1| 2024-01-01 00:36:38|                 N|
|       1| 2024-01-01 00:46:51|                 N|
+--------+--------------------+------------------+
only showing top 5 rows



In [8]:
df_parq.select('store_and_fwd_flag').distinct().show(5)

+------------------+
|store_and_fwd_flag|
+------------------+
|                 Y|
|                 N|
|              NULL|
+------------------+



In [9]:
df_parq.dtypes
df_parq.select('VendorID','tpep_pickup_datetime').dtypes

[('VendorID', 'int'), ('tpep_pickup_datetime', 'timestamp_ntz')]

In [10]:
df_parq.describe().show()

+-------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+------------------+--------------------+-------------------+
|summary|          VendorID|   passenger_count|     trip_distance|       RatecodeID|store_and_fwd_flag|      PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+------------------+--------------------+----

# Transformation Steps

In [11]:
#Add column
# A simple add is df.withColumn('New Column Name',derived column_value).col references a dataframe column that is called
df_parq.withColumn("Vendor_name",
    when(col("VendorID") == 1, "Creative Mobile Technologies, LLC")
    .when(col("VendorID") == 2, "Curb Mobility, LLC")
    .when(col("VendorID") == 6, "Myle Technologies Inc")
    .when(col("VendorID") == 7, "Helix")
    .otherwise("No Vendor")
)\
.withColumn(
        "store_and_forward_trip_flag",
        coalesce(regexp_replace(col("store_and_fwd_flag"), '"', ''), lit("N/A"))
).show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+---------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|       Vendor_name|store_and_forward_trip_flag|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+---------------------------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01

In [12]:
# Use sql case when inside Pyspark

## Add additional columns. To add multiple columns just continue adding .withColumn after each column. \ specifies a new line

df_parq = df_parq\
.withColumn(
    "vendor_name",
    expr("""
        case
            when vendorID = 1 then 'Creative Mobile Technologies, LLC'
            when VendorID = 2 then 'Curb Mobility, LLC'
            when VendorID = 6 then 'Myle Technologies Inc'
            when VendorID = 7 then 'Helix'
            else 'No Vendor'
        end
    """)
)\
.withColumn(
    "rate_code_type",
    expr("""
        case 
            when ratecodeid = 1 then 'Standard rate'
            when ratecodeid = 2 then 'JFK'
            when ratecodeid = 3 then 'Newark'
            when ratecodeid = 4 then 'Nassau or Westchester'
            when ratecodeid = 5 then 'Negotiated fare'
            when ratecodeid = 6 then 'Group ride'
            when ratecodeid = 99 then 'Unknown'
            else 'N/A'
        end
    """)
)\
.withColumn(
    "payment_type", #since column name already exists, it overwrites it
    expr("""
        case
            when payment_type = 0 then 'Flex Fare trip'
            when payment_type = 1 then 'Credit card'
            when payment_type = 2 then 'Cash'
            when payment_type = 3 then 'No charge'
            when payment_type = 4 then 'Dispute'
            when payment_type = 5 then 'Unknown'
            when payment_type = 6 then 'Voided trip'
            else 'N/A'
        end
    """)
)


In [13]:
#Add forgotten column
df_parq = df_parq\
.withColumn("trip_duration_mins", 
            expr("(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 60")
)\
.withColumn("store_and_forward_trip_flag",
            expr("coalesce(regexp_replace(store_and_fwd_flag, '\"', ''), 'N/A')")
)

In [14]:
df_parq.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+---------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|         vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+-

In [15]:
df_parq.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: string (nullable = false)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- vendor_name: string (nullable = false)
 |-- rate_code_type: string (nullable = false)
 |-- trip_duration_mins: double 

In [16]:
##Drop Columns
df_parq=df_parq.drop('store_and_fwd_flag')

In [17]:
df_parq.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------+------------------+---------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|       vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------+------------------+---------------------------+
|

In [18]:
## Rename columns
#df_parq.withColumnRenamed('Airport_fee','airport_fee')

In [19]:
##Rename multiple columns
df_parq=df_parq\
    .withColumnRenamed('VendorID','vendor_id').withColumnRenamed('RatecodeID','rate_code_id')\
    .withColumnRenamed('tpep_pickup_datetime','pickup_time').withColumnRenamed('tpep_dropoff_datetime','dropoff_time')\
    .withColumnRenamed('DOLocationID','dropoff_zone').withColumnRenamed('PULocationID','pickup_zone')\
    .withColumnRenamed('Airport_fee','airport_fee').withColumnRenamed('extra','extra_fees')

In [20]:
df_parq.show(2)

+---------+-------------------+-------------------+---------------+-------------+------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+---------------------------+
|vendor_id|        pickup_time|       dropoff_time|passenger_count|trip_distance|rate_code_id|pickup_zone|dropoff_zone|payment_type|fare_amount|extra_fees|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|         vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+---------+-------------------+-------------------+---------------+-------------+------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+------------

In [21]:
# reorder columns
cols = [
        "vendor_id","vendor_name","pickup_time","dropoff_time","trip_duration_mins","passenger_count","trip_distance"
        ,"rate_code_id","rate_code_type","store_and_forward_trip_flag","pickup_zone","dropoff_zone","payment_type"
        ,"fare_amount","extra_fees","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount"
        ,"congestion_surcharge","airport_fee"
       ]
df_parq= df_parq.select(cols)

In [22]:
df_parq.show(3)

+---------+--------------------+-------------------+-------------------+------------------+---------------+-------------+------------+--------------+---------------------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+
|vendor_id|         vendor_name|        pickup_time|       dropoff_time|trip_duration_mins|passenger_count|trip_distance|rate_code_id|rate_code_type|store_and_forward_trip_flag|pickup_zone|dropoff_zone|payment_type|fare_amount|extra_fees|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+---------+--------------------+-------------------+-------------------+------------------+---------------+-------------+------------+--------------+---------------------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+-----------------

In [23]:
# filter can be done both ways

#df_parq.filter('trip_distance <= 0').show()
df_parq.filter(df_parq['trip_distance'] <= 0).show(1)

+---------+------------------+-------------------+-------------------+-------------------+---------------+-------------+------------+--------------+---------------------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+
|vendor_id|       vendor_name|        pickup_time|       dropoff_time| trip_duration_mins|passenger_count|trip_distance|rate_code_id|rate_code_type|store_and_forward_trip_flag|pickup_zone|dropoff_zone|payment_type|fare_amount|extra_fees|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+---------+------------------+-------------------+-------------------+-------------------+---------------+-------------+------------+--------------+---------------------------+-----------+------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------

In [24]:
#select only necessary rows
df_parq.filter('trip_distance <= 0').select(['vendor_name','trip_distance']).show() 

+--------------------+-------------+
|         vendor_name|trip_distance|
+--------------------+-------------+
|  Curb Mobility, LLC|          0.0|
|Creative Mobile T...|          0.0|
|Creative Mobile T...|          0.0|
|Creative Mobile T...|          0.0|
|Creative Mobile T...|          0.0|
|Creative Mobile T...|          0.0|
|  Curb Mobility, LLC|          0.0|
|Creative Mobile T...|          0.0|
|Creative Mobile T...|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
|Creative Mobile T...|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
|Creative Mobile T...|          0.0|
|  Curb Mobility, LLC|          0.0|
|  Curb Mobility, LLC|          0.0|
+--------------------+-------------+
only showing top 20 rows



In [25]:
#multiple filter operations --& for and | for or 
df_parq.filter( (df_parq['trip_distance'] <= 0) & (df_parq['tip_amount'] > 0)  ).\
select('vendor_name','pickup_time','dropoff_time','trip_distance','tip_amount','fare_amount','payment_type').show()


+--------------------+-------------------+-------------------+-------------+----------+-----------+------------+
|         vendor_name|        pickup_time|       dropoff_time|trip_distance|tip_amount|fare_amount|payment_type|
+--------------------+-------------------+-------------------+-------------+----------+-----------+------------+
|Creative Mobile T...|2024-01-01 00:58:50|2024-01-01 01:01:10|          0.0|       2.0|        4.4| Credit card|
|Creative Mobile T...|2024-01-01 00:15:16|2024-01-01 00:26:58|          0.0|       3.1|       10.7| Credit card|
|Creative Mobile T...|2024-01-01 00:39:34|2024-01-01 01:04:02|          0.0|       2.0|       19.8| Credit card|
|Creative Mobile T...|2024-01-01 00:25:58|2024-01-01 00:27:19|          0.0|       2.2|       10.0| Credit card|
|  Curb Mobility, LLC|2024-01-01 00:28:27|2024-01-01 00:29:15|          0.0|      36.2|      180.0| Credit card|
|Creative Mobile T...|2024-01-01 00:31:06|2024-01-01 00:42:00|          0.0|       8.0|        3

In [26]:
df_parq.count()

2964624

In [27]:
#invalid trip data to be stored for data quality checks
df_invalid_tripdata = df_parq.filter((df_parq['trip_distance'] <= 0) | (df_parq['fare_amount'] <= 0) | (df_parq['passenger_count'] <= 0) )

In [28]:
df_invalid_tripdata.select('vendor_name','pickup_time','dropoff_time','trip_distance','tip_amount','fare_amount','passenger_count').show(5)

+--------------------+-------------------+-------------------+-------------+----------+-----------+---------------+
|         vendor_name|        pickup_time|       dropoff_time|trip_distance|tip_amount|fare_amount|passenger_count|
+--------------------+-------------------+-------------------+-------------+----------+-----------+---------------+
|Creative Mobile T...|2024-01-01 00:30:40|2024-01-01 00:58:40|          3.0|       0.0|       25.4|              0|
|  Curb Mobility, LLC|2024-01-01 00:52:09|2024-01-01 00:52:28|          0.0|       0.0|        3.0|              1|
|Creative Mobile T...|2024-01-01 00:14:29|2024-01-01 00:14:29|          0.0|       0.0|        3.0|              1|
|  Curb Mobility, LLC|2024-01-01 00:18:24|2024-01-01 00:30:39|         2.16|       0.0|      -13.5|              1|
|Creative Mobile T...|2024-01-01 00:58:50|2024-01-01 01:01:10|          0.0|       2.0|        4.4|              1|
+--------------------+-------------------+-------------------+----------

In [29]:
# Option 1 to exclude invalid trip data and only work with valid trips. This performs set difference, doesn't exclude nulls, expensive and not best practice for ETL purpose
#df_clean1=df_parq.subtract(df_invalid_tripdata) #remove invalid data from whole dataset 

# Option 2 filters only what is needed and handles null properly
df_clean=df_parq.filter( (df_parq['trip_distance'] > 0) & (df_parq['fare_amount'] > 0) & (df_parq['passenger_count'] > 0) )


In [30]:
# Validation check 
df_clean.filter((df_parq['trip_distance'] <= 0) | (df_parq['fare_amount'] <= 0) | (df_parq['passenger_count'] <= 0) )\
.select('vendor_name','pickup_time','dropoff_time','trip_distance','fare_amount','passenger_count').count()

0

In [31]:
#flag anomalies and log into another dataframe
df_clean.filter((df_parq['trip_duration_mins'] > 180) & (df_parq['trip_distance'] > 100) & (df_parq['fare_amount'] > 500))\
.select('vendor_name','pickup_time','dropoff_time','trip_distance','fare_amount','trip_duration_mins').show()

+--------------------+-------------------+-------------------+-------------+-----------+------------------+
|         vendor_name|        pickup_time|       dropoff_time|trip_distance|fare_amount|trip_duration_mins|
+--------------------+-------------------+-------------------+-------------+-----------+------------------+
|  Curb Mobility, LLC|2024-01-02 07:50:08|2024-01-02 11:29:29|       233.25|     1616.5|            219.35|
|Creative Mobile T...|2024-01-16 19:00:33|2024-01-16 23:07:48|        210.2|      650.0|            247.25|
|  Curb Mobility, LLC|2024-01-17 10:46:15|2024-01-17 13:48:11|       109.75|      761.1|181.93333333333334|
|Creative Mobile T...|2024-01-26 12:49:27|2024-01-26 16:01:22|        153.2|      550.4|191.91666666666666|
+--------------------+-------------------+-------------------+-------------+-----------+------------------+



In [32]:
from pyspark.sql.functions import array, when, col, size, lit

anomalies_data = df_clean.withColumn(
    "anomaly_type",
    array(
        when(col("fare_amount") > 500, lit("high_fare")) ,
        when(col("trip_distance") > 100, lit("long_distance")),
        when(col("trip_duration_mins") > 180,lit( "long_duration")),
        when(col("passenger_count") > 6, lit("high_passenger_count"))
    ) #returns an array of the type of anomaly for each condition
).withColumn("anomaly_type",
    expr("filter(anomaly_type, x -> x is not null)") #removes null anomaly to focus on actual anomalies
).filter(
    size(col("anomaly_type")) > 0 #only retains those with anomaly
)


In [33]:
anomalies_data.filter(size(col("anomaly_type")) > 1).select('vendor_name','trip_distance','fare_amount','trip_duration_mins','anomaly_type').show(truncate=False)

+---------------------------------+-------------+-----------+------------------+-----------------------------------------+
|vendor_name                      |trip_distance|fare_amount|trip_duration_mins|anomaly_type                             |
+---------------------------------+-------------+-----------+------------------+-----------------------------------------+
|Curb Mobility, LLC               |233.25       |1616.5     |219.35            |[high_fare, long_distance, long_duration]|
|Curb Mobility, LLC               |142.62       |912.3      |160.1             |[high_fare, long_distance]               |
|Curb Mobility, LLC               |111.57       |678.5      |114.38333333333334|[high_fare, long_distance]               |
|Curb Mobility, LLC               |210.82       |500.0      |186.46666666666667|[long_distance, long_duration]           |
|Curb Mobility, LLC               |31.95        |2221.3     |3106.1833333333334|[high_fare, long_duration]               |
|Curb Mobility, 

## Get Taxi Zone Lookup to further enrich data

In [34]:
import time 
import os
import requests
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.csv as pc
from datetime import datetime

In [ ]:
#read the files directly from download without saving
url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
taxi_zones = pd.read_csv(url)
taxi_zones.head()

In [35]:
#download from a link and save scv
url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
folder = 'C:/Users/User/Documents/PORTFOLIO/NYC TAXI DATA/taxi_zones_lookup'
filename = 'taxi_zones_lookup.csv'
os.makedirs(folder, exist_ok=True) #create folder in specified directory

file_path = os.path.join(folder, filename)
r = requests.get(url, stream=True, timeout=60) 
with open(file_path, "wb") as f:
    f.write(r.content)

In [ ]:
#read csv from folder with pandas
#df = pd.read_csv('C:/Users/User/Documents/PORTFOLIO/NYC TAXI DATA/taxi_zones_lookup/taxi_zones_lookup.csv')
file_location = f'{folder}/{filename}'
#print(file_location)
taxi_zones = pd.read_csv(file_location)
taxi_zones.head()

In [36]:
# read csv in spark
file_location = f'{folder}/{filename}'
taxi_zones=spark.read.csv(file_location
    ,header=True #sets the first row to be the header
    ,inferSchema = True #Default is string if infer Schema is not specified.Ensures to infer the correct datatype of columns. 
)


In [37]:
taxi_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [38]:
df_clean.select('vendor_name','pickup_zone','dropoff_zone').show(5,truncate=False)

+---------------------------------+-----------+------------+
|vendor_name                      |pickup_zone|dropoff_zone|
+---------------------------------+-----------+------------+
|Curb Mobility, LLC               |186        |79          |
|Creative Mobile Technologies, LLC|140        |236         |
|Creative Mobile Technologies, LLC|236        |79          |
|Creative Mobile Technologies, LLC|79         |211         |
|Creative Mobile Technologies, LLC|211        |148         |
+---------------------------------+-----------+------------+
only showing top 5 rows



In [40]:
#Join the taxi zones and drop location ID
taxi_zones_pickup = taxi_zones.alias('a')
taxi_zones_dropoff =taxi_zones.alias('b')

df_clean=df_clean\
.join(taxi_zones_pickup,df_clean.pickup_zone ==taxi_zones_pickup.LocationID, 'left')\
.withColumnRenamed("Zone", "pickup_zone_name") \
.withColumnRenamed("Borough", "pickup_borough_name")\
.withColumnRenamed("service_zone", "pickup_service_zone")\
.join(taxi_zones_dropoff,df_clean.dropoff_zone ==taxi_zones_dropoff.LocationID, 'left')\
.withColumnRenamed("Zone", "dropoff_zone_name") \
.withColumnRenamed("Borough", "dropoff_borough_name")\
.withColumnRenamed("service_zone", "dropoff_service_zone").drop('locationid')


In [44]:
df_clean.select(
    "pickup_zone", "pickup_zone_name", "pickup_borough_name","pickup_service_zone",
    "dropoff_zone", "dropoff_zone_name","dropoff_borough_name", "dropoff_service_zone"
).show(5,truncate=False)

+-----------+----------------------------+-------------------+-------------------+------------+---------------------+--------------------+--------------------+
|pickup_zone|pickup_zone_name            |pickup_borough_name|pickup_service_zone|dropoff_zone|dropoff_zone_name    |dropoff_borough_name|dropoff_service_zone|
+-----------+----------------------------+-------------------+-------------------+------------+---------------------+--------------------+--------------------+
|186        |Penn Station/Madison Sq West|Manhattan          |Yellow Zone        |79          |East Village         |Manhattan           |Yellow Zone         |
|140        |Lenox Hill East             |Manhattan          |Yellow Zone        |236         |Upper East Side North|Manhattan           |Yellow Zone         |
|236        |Upper East Side North       |Manhattan          |Yellow Zone        |79          |East Village         |Manhattan           |Yellow Zone         |
|79         |East Village               

## Other Transformation functions for knowledge base ...Not required for this project

In [ ]:
#Drop rows
#df_parq.select('trip_distance').distinct().show()

In [ ]:
#df.na.drop(how="any",thresh=2,subset=['columnname']).show()
parameters inside drop()
how - can accept 2 values:
    any - drop rows with any values as null
    all - drop rows with all values as null
thresh - specifies the threshold of non null values required to be present
    if thresh is set to 2, it deletes rows where less than 2 non null values appear
subset - deletes null values in the specified column


In [ ]:
df_parq.selectExpr(
    *[f"count({c}) as {c}" for c in df_parq.columns]
).show() #count values in each column


In [ ]:
df_parq.filter(col("trip_distance").isNull()).show(5)
#df_parq.filter(col("passenger_count").isNull()).count() #count null values in column
#df_drop_test = df_parq.na.drop(how="any",thresh=20)


In [ ]:
#df_drop_test.filter(col("passenger_count").isNull()).count()
df_drop_test.selectExpr(
    *[f"count({c}) as {c}" for c in df_drop_test.columns]
).show() #count values in each column


In [ ]:
# filling missing values
#df.na.fill('updatedvalue',subset)
df_parq.select('passenger_count').distinct().show()

In [ ]:
df=df_parq.na.fill('Unknown','passenger_count')

In [ ]:
df.select('passenger_count').distinct().show()